In [1]:
import os
import glob
import pandas as pd
import numpy as np
import sys
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from sklearn.metrics import confusion_matrix
import pickle

In [2]:
# total ver.
# load dataset (pickle files)

input_path1 = 'C:/Users/LG/Desktop/frost_data/ver3/SMOTE/1class'
input_path2 = 'C:/Users/LG/Desktop/frost_data/ver3/ML_test'
input_path3 = 'C:/Users/LG/Desktop/frost_data/ver3/ML_vali'
input_path4 = 'C:/Users/LG/Desktop/frost_data/ver3/ML_train'
for i in [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12]:
    with open(f'{input_path2}/site{i}.pkl','rb') as f3:
        test = pickle.load(f3)
    with open(f'{input_path4}/site{i}.pkl','rb') as f5:
        train = pickle.load(f5)              
    if i == 1:
        total_te = test
        total_tr = train
    else:
        total_te = np.vstack((total_te,test))
        total_tr = np.vstack((total_tr,train))
for i in [13, 14]:
    with open(f'{input_path3}/site{i}.pkl','rb') as f4:
        vali = pickle.load(f4)
    if i == 13:
        total_va = vali
    else:
        total_va = np.vstack((total_va,vali))

with open(f'{input_path1}/X_train.pkl','rb') as f1:
    X_train = pickle.load(f1)
with open(f'{input_path1}/Y_train.pkl','rb') as f2:
    Y_train = pickle.load(f2)   
with open(f'{input_path1}/scaler.pkl','rb') as f3:
    scaler = pickle.load(f3)
X_test = total_te[:,:-2]
X_test = scaler.transform(X_test)
X_vali = total_va[:,:-2]
X_vali = scaler.transform(X_vali)
X_train2 = total_tr[:,:-2]
X_train2 = scaler.transform(X_train2)
Y_test = total_te[:,-2]
Y_vali = total_va[:,-2]
Y_train = Y_train.T
Y_train2 = total_tr[:,-2]
print('Finish setting data')

Finish setting data


In [3]:
#clf = RandomForestClassifier(n_estimators=130, criterion = 'gini', max_depth=1
#                             , random_state = 0, min_samples_split = 2, min_samples_leaf = 1, max_leaf_nodes = None)
clf = RandomForestClassifier(n_estimators=130, max_depth=4, random_state = 0)
model = clf.fit(X_train, Y_train)

In [4]:
f_import = model.feature_importances_
print(f_import)

[0.24106281 0.1103641  0.02024987 0.14118212 0.03768641 0.04111913
 0.29983856 0.10849698]


In [5]:
test_pb = model.predict(X_test)
vali_pb = model.predict(X_vali)
train_pb = model.predict(X_train2)

In [6]:
i = 0
j = 0
ck_hour = 1
while i + ck_hour < test_pb.size:
    if 0 == sum(test_pb[i:i+ck_hour]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_test[i:i+ck_hour]):
        real_ck = 0
    else:
        real_ck = 1
    i += ck_hour
    if j == 0:
        test_Y = frost_ck
        t_real_Y = real_ck
        j += 1
    else:
        test_Y = np.hstack((test_Y,frost_ck))
        t_real_Y = np.hstack((t_real_Y,real_ck))

i = 0
j = 0
while i + ck_hour < vali_pb.size:
    if 0 == sum(vali_pb[i:i+ck_hour]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_vali[i:i+ck_hour]):
        real_ck = 0
    else:
        real_ck = 1
    i += ck_hour
    if j == 0:
        vali_Y = frost_ck
        v_real_Y = real_ck
        j += 1
    else:
        vali_Y = np.hstack((vali_Y,frost_ck))
        v_real_Y = np.hstack((v_real_Y,real_ck))
i = 0
j = 0
while i + ck_hour < train_pb.size:
    if 0 == sum(train_pb[i:i+ck_hour]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_train2[i:i+ck_hour]):
        real_ck = 0
    else:
        real_ck = 1
    i += ck_hour
    if j == 0:
        train_Y = frost_ck
        tr_real_Y = real_ck
        j += 1
    else:
        train_Y = np.hstack((train_Y,frost_ck))
        tr_real_Y = np.hstack((tr_real_Y,real_ck))
cfTrain =confusion_matrix(tr_real_Y,train_Y)
TN1 = cfTrain[0,0]
FP1 = cfTrain[0,1]
FN1 = cfTrain[1,0]
TP1 = cfTrain[1,1]
Pre1 = TP1/(TP1+FP1) #Precision
Rec1 = TP1/(TP1+FN1) #Recall
F1_1 = 2*(Pre1*Rec1)/(Pre1+Rec1) # F1-score
cfTest =confusion_matrix(t_real_Y,test_Y)
TN2 = cfTest[0,0]
FP2 = cfTest[0,1]
FN2 = cfTest[1,0]
TP2 = cfTest[1,1]
Pre2 = TP2/(TP2+FP2) #Precision
Rec2 = TP2/(TP2+FN2) #Recall
F1_2 = 2*(Pre2*Rec2)/(Pre2+Rec2) # F1-score
cfVali =confusion_matrix(v_real_Y,vali_Y)
TN3 = cfVali[0,0]
FP3 = cfVali[0,1]
FN3 = cfVali[1,0]
TP3 = cfVali[1,1]
Pre3 = TP3/(TP3+FP3) #Precision
Rec3 = TP3/(TP3+FN3) #Recall
F1_3 = 2*(Pre3*Rec3)/(Pre2+Rec3) # F1-score
acc3 = (TP3+TN3)/(TN3+FP3+FN3+TP3)
csi3 = TP3/(TP3+FP3+FN3)
print(cfTrain)
print(cfTest)
print(cfVali)
#another score
acc1 = (TP1+TN1)/(TN1+FP1+FN1+TP1)
csi1 = TP1/(TP1+FP1+FN1)
acc2 = (TP2+TN2)/(TN2+FP2+FN2+TP2)
csi2 = TP2/(TP2+FP2+FN2)
spc1 = TN1/(TN1+FP1)
spc2 = TN2/(TN2+FP2)
spc3 = TN3/(TN3+FP3)
print("--------- Train check -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}, spc:{:2f}".format(F1_1,acc1,Pre1,Rec1,csi1,spc1))
print("--------- Verification -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}, spc:{:2f}".format(F1_2,acc2,Pre2,Rec2,csi2,spc2))
print("--------- Validation -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}, spc:{:2f}".format(F1_3,acc3,Pre3,Rec3,csi3,spc3))

[[260047 156738]
 [ 13553  44234]]
[[244863  78110]
 [ 69399  78988]]
[[53869 17877]
 [ 5501  8454]]
--------- Train check -----------
F1:0.341893, acc:0.641169, pre:0.220100, rec:0.765466, csi:0.206195, spc:0.623936
--------- Verification -----------
F1:0.517132, acc:0.687057, pre:0.502794, rec:0.532311, csi:0.348738, spc:0.758153
--------- Validation -----------
F1:0.350900, acc:0.727214, pre:0.321066, rec:0.605804, csi:0.265582, spc:0.750829


In [7]:
# all class ver.
# load dataset (pickle files)

input_path1 = 'C:/Users/LG/Desktop/frost_data/ver3/SMOTE/73class'
input_path2 = 'C:/Users/LG/Desktop/frost_data/ver3/ML_test'
input_path3 = 'C:/Users/LG/Desktop/frost_data/ver3/ML_vali'
input_path4 = 'C:/Users/LG/Desktop/frost_data/ver3/ML_train'
for i in [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12]:
    with open(f'{input_path2}/site{i}.pkl','rb') as f3:
        test = pickle.load(f3)
    with open(f'{input_path4}/site{i}.pkl','rb') as f5:
        train = pickle.load(f5)               
    if i == 1:
        total_te = test
        total_tr = train
    else:
        total_te = np.vstack((total_te,test))
        total_tr = np.vstack((total_tr,train))
for i in [13, 14]:
    with open(f'{input_path3}/site{i}.pkl','rb') as f4:
        vali = pickle.load(f4)
    if i == 13:
        total_va = vali
    else:
        total_va = np.vstack((total_va,vali))



for i in range(0,73):
    with open(f'{input_path1}/X_train_T{i}.pkl','rb') as f1:
        X_train_od = pickle.load(f1)
    with open(f'{input_path1}/Y_train_T{i}.pkl','rb') as f2:
        Y_train_od = pickle.load(f2) 
    if i == 0:
        X_train = X_train_od
        Y_train = Y_train_od
    else:
        X_train = np.vstack((X_train,X_train_od))
        Y_train = np.hstack((Y_train,Y_train_od))
with open(f'C:/Users/LG/Desktop/frost_data/ver3/SMOTE/1class/scaler.pkl','rb') as f3:
    scaler = pickle.load(f3)
X_test = total_te[:,:-2]
X_test = scaler.transform(X_test)
X_vali = total_va[:,:-2]
X_vali = scaler.transform(X_vali)
X_train2 = total_tr[:,:-2]
X_train2 = scaler.transform(X_train2)
Y_test = total_te[:,-2]
Y_vali = total_va[:,-2]
Y_train = Y_train.T
Y_train2 = total_tr[:,-2]
print('Finish setting data')

Finish setting data


In [8]:
clf = RandomForestClassifier(n_estimators=130, max_depth=3, random_state = 0)
model = clf.fit(X_train, Y_train)

In [9]:
test_pb = model.predict(X_test)
vali_pb = model.predict(X_vali)
train_pb = model.predict(X_train2)

In [10]:
i = 0
j = 0
ck_hour = 1
while i + ck_hour < test_pb.size:
    if 0 == sum(test_pb[i:i+ck_hour]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_test[i:i+ck_hour]):
        real_ck = 0
    else:
        real_ck = 1
    i += ck_hour
    if j == 0:
        test_Y = frost_ck
        t_real_Y = real_ck
        j += 1
    else:
        test_Y = np.hstack((test_Y,frost_ck))
        t_real_Y = np.hstack((t_real_Y,real_ck))

i = 0
j = 0
while i + ck_hour < vali_pb.size:
    if 0 == sum(vali_pb[i:i+ck_hour]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_vali[i:i+ck_hour]):
        real_ck = 0
    else:
        real_ck = 1
    i += ck_hour
    if j == 0:
        vali_Y = frost_ck
        v_real_Y = real_ck
        j += 1
    else:
        vali_Y = np.hstack((vali_Y,frost_ck))
        v_real_Y = np.hstack((v_real_Y,real_ck))
i = 0
j = 0
while i + ck_hour < train_pb.size:
    if 0 == sum(train_pb[i:i+ck_hour]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_train2[i:i+ck_hour]):
        real_ck = 0
    else:
        real_ck = 1
    i += ck_hour
    if j == 0:
        train_Y = frost_ck
        tr_real_Y = real_ck
        j += 1
    else:
        train_Y = np.hstack((train_Y,frost_ck))
        tr_real_Y = np.hstack((tr_real_Y,real_ck))
cfTrain =confusion_matrix(tr_real_Y,train_Y)
TN1 = cfTrain[0,0]
FP1 = cfTrain[0,1]
FN1 = cfTrain[1,0]
TP1 = cfTrain[1,1]
Pre1 = TP1/(TP1+FP1) #Precision
Rec1 = TP1/(TP1+FN1) #Recall
F1_1 = 2*(Pre1*Rec1)/(Pre1+Rec1) # F1-score
cfTest =confusion_matrix(t_real_Y,test_Y)
TN2 = cfTest[0,0]
FP2 = cfTest[0,1]
FN2 = cfTest[1,0]
TP2 = cfTest[1,1]
Pre2 = TP2/(TP2+FP2) #Precision
Rec2 = TP2/(TP2+FN2) #Recall
F1_2 = 2*(Pre2*Rec2)/(Pre2+Rec2) # F1-score
cfVali =confusion_matrix(v_real_Y,vali_Y)
TN3 = cfVali[0,0]
FP3 = cfVali[0,1]
FN3 = cfVali[1,0]
TP3 = cfVali[1,1]
Pre3 = TP3/(TP3+FP3) #Precision
Rec3 = TP3/(TP3+FN3) #Recall
F1_3 = 2*(Pre3*Rec3)/(Pre2+Rec3) # F1-score
acc3 = (TP3+TN3)/(TN3+FP3+FN3+TP3)
csi3 = TP3/(TP3+FP3+FN3)
print(cfTrain)
print(cfTest)
print(cfVali)
#another score
acc1 = (TP1+TN1)/(TN1+FP1+FN1+TP1)
csi1 = TP1/(TP1+FP1+FN1)
acc2 = (TP2+TN2)/(TN2+FP2+FN2+TP2)
csi2 = TP2/(TP2+FP2+FN2)
spc1 = TN1/(TN1+FP1)
spc2 = TN2/(TN2+FP2)
spc3 = TN3/(TN3+FP3)
print("--------- Train check -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}, spc:{:2f}".format(F1_1,acc1,Pre1,Rec1,csi1,spc1))
print("--------- Verification -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}, spc:{:2f}".format(F1_2,acc2,Pre2,Rec2,csi2,spc2))
print("--------- Validation -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}, spc:{:2f}".format(F1_3,acc3,Pre3,Rec3,csi3,spc3))

[[238168 178617]
 [ 13542  44245]]
[[223373  99600]
 [ 53110  95277]]
[[45658 26088]
 [ 3616 10339]]
--------- Train check -----------
F1:0.315305, acc:0.595090, pre:0.198531, rec:0.765657, csi:0.187158, spc:0.571441
--------- Verification -----------
F1:0.555124, acc:0.676023, pre:0.488908, rec:0.642085, csi:0.384202, spc:0.691615
--------- Validation -----------
F1:0.341982, acc:0.653400, pre:0.283828, rec:0.740881, csi:0.258197, spc:0.636384


In [11]:
# 12class ver.
# load dataset (pickle files)

input_path1 = 'C:/Users/LG/Desktop/frost_data/ver3/SMOTE/12class'
cut_i = [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72]
j = 0
for i in range(1,13):
    with open(f'{input_path1}/X_train_T{i}.pkl','rb') as f1:
        X_train_od = pickle.load(f1)
    with open(f'{input_path1}/Y_train_T{i}.pkl','rb') as f2:
        Y_train_od = pickle.load(f2) 
    if i == 1:
        X_train = X_train_od
        Y_train = Y_train_od
    else:
        X_train = np.vstack((X_train,X_train_od))
        Y_train = np.hstack((Y_train,Y_train_od))


Y_train = Y_train.T

In [12]:
clf = RandomForestClassifier(n_estimators=130, max_depth=4, random_state = 0)
model = clf.fit(X_train, Y_train)
test_pb = model.predict(X_test)
vali_pb = model.predict(X_vali)
train_pb = model.predict(X_train2)

In [14]:
i = 0
j = 0
ck_hour = 1
while i + ck_hour < test_pb.size:
    if 0 == sum(test_pb[i:i+ck_hour]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_test[i:i+ck_hour]):
        real_ck = 0
    else:
        real_ck = 1
    i += ck_hour
    if j == 0:
        test_Y = frost_ck
        t_real_Y = real_ck
        j += 1
    else:
        test_Y = np.hstack((test_Y,frost_ck))
        t_real_Y = np.hstack((t_real_Y,real_ck))

i = 0
j = 0
while i + ck_hour < vali_pb.size:
    if 0 == sum(vali_pb[i:i+ck_hour]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_vali[i:i+ck_hour]):
        real_ck = 0
    else:
        real_ck = 1
    i += ck_hour
    if j == 0:
        vali_Y = frost_ck
        v_real_Y = real_ck
        j += 1
    else:
        vali_Y = np.hstack((vali_Y,frost_ck))
        v_real_Y = np.hstack((v_real_Y,real_ck))
i = 0
j = 0
while i + ck_hour < train_pb.size:
    if 0 == sum(train_pb[i:i+ck_hour]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_train2[i:i+ck_hour]):
        real_ck = 0
    else:
        real_ck = 1
    i += ck_hour
    if j == 0:
        train_Y = frost_ck
        tr_real_Y = real_ck
        j += 1
    else:
        train_Y = np.hstack((train_Y,frost_ck))
        tr_real_Y = np.hstack((tr_real_Y,real_ck))
cfTrain =confusion_matrix(tr_real_Y,train_Y)
TN1 = cfTrain[0,0]
FP1 = cfTrain[0,1]
FN1 = cfTrain[1,0]
TP1 = cfTrain[1,1]
Pre1 = TP1/(TP1+FP1) #Precision
Rec1 = TP1/(TP1+FN1) #Recall
F1_1 = 2*(Pre1*Rec1)/(Pre1+Rec1) # F1-score
cfTest =confusion_matrix(t_real_Y,test_Y)
TN2 = cfTest[0,0]
FP2 = cfTest[0,1]
FN2 = cfTest[1,0]
TP2 = cfTest[1,1]
Pre2 = TP2/(TP2+FP2) #Precision
Rec2 = TP2/(TP2+FN2) #Recall
F1_2 = 2*(Pre2*Rec2)/(Pre2+Rec2) # F1-score
cfVali =confusion_matrix(v_real_Y,vali_Y)
TN3 = cfVali[0,0]
FP3 = cfVali[0,1]
FN3 = cfVali[1,0]
TP3 = cfVali[1,1]
Pre3 = TP3/(TP3+FP3) #Precision
Rec3 = TP3/(TP3+FN3) #Recall
F1_3 = 2*(Pre3*Rec3)/(Pre2+Rec3) # F1-score
acc3 = (TP3+TN3)/(TN3+FP3+FN3+TP3)
csi3 = TP3/(TP3+FP3+FN3)
print(cfTrain)
print(cfTest)
print(cfVali)
#another score
acc1 = (TP1+TN1)/(TN1+FP1+FN1+TP1)
csi1 = TP1/(TP1+FP1+FN1)
acc2 = (TP2+TN2)/(TN2+FP2+FN2+TP2)
csi2 = TP2/(TP2+FP2+FN2)
spc1 = TN1/(TN1+FP1)
spc2 = TN2/(TN2+FP2)
spc3 = TN3/(TN3+FP3)
print("--------- Train check -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}, spc:{:2f}".format(F1_1,acc1,Pre1,Rec1,csi1,spc1))
print("--------- Verification -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}, spc:{:2f}".format(F1_2,acc2,Pre2,Rec2,csi2,spc2))
print("--------- Validation -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}, spc:{:2f}".format(F1_3,acc3,Pre3,Rec3,csi3,spc3))

[[243900 172885]
 [ 12094  45693]]
[[227738  95235]
 [ 56999  91388]]
[[49233 22513]
 [ 4417  9538]]
--------- Train check -----------
F1:0.330671, acc:0.610219, pre:0.209047, rec:0.790714, csi:0.198086, spc:0.585194
--------- Verification -----------
F1:0.545584, acc:0.677032, pre:0.489693, rec:0.615876, csi:0.375122, spc:0.705130
--------- Validation -----------
F1:0.346745, acc:0.685768, pre:0.297588, rec:0.683483, csi:0.261544, spc:0.686212


In [28]:
# 3class ver.
# load dataset (pickle files)

input_path1 = 'C:/Users/LG/Desktop/frost_data/new3/3class/SMOTE'
cut_i = [0, 24, 48, 72]
j = 0
for i in range(1,4):
    with open(f'{input_path1}/X_train_T{i}.pkl','rb') as f1:
        X_train_od = pickle.load(f1)
    with open(f'{input_path1}/Y_train_T{i}.pkl','rb') as f2:
        Y_train_od = pickle.load(f2) 
    if i == 1:
        X_train = X_train_od
        Y_train = Y_train_od
    else:
        X_train = np.vstack((X_train,X_train_od))
        Y_train = np.hstack((Y_train,Y_train_od))

X_train = np.delete(X_train, 1, axis = 1)
Y_train = Y_train.T

In [39]:
clf = RandomForestClassifier(n_estimators=130, max_depth=5, random_state = 0)
model = clf.fit(X_train, Y_train)
test_pb = model.predict(X_test)
vali_pb = model.predict(X_vali)
train_pb = model.predict(X_train2)

In [40]:
i = 0
j = 0
while i + 12 < test_pb.size:
    if 0 == sum(test_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_test[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        test_Y = frost_ck
        t_real_Y = real_ck
        j += 1
    else:
        test_Y = np.hstack((test_Y,frost_ck))
        t_real_Y = np.hstack((t_real_Y,real_ck))

i = 0
j = 0
while i + 12 < vali_pb.size:
    if 0 == sum(vali_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_vali[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        vali_Y = frost_ck
        v_real_Y = real_ck
        j += 1
    else:
        vali_Y = np.hstack((vali_Y,frost_ck))
        v_real_Y = np.hstack((v_real_Y,real_ck))
i = 0
j = 0
while i + 12 < train_pb.size:
    if 0 == sum(train_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_train2[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        train_Y = frost_ck
        tr_real_Y = real_ck
        j += 1
    else:
        train_Y = np.hstack((train_Y,frost_ck))
        tr_real_Y = np.hstack((tr_real_Y,real_ck))
cfTrain =confusion_matrix(tr_real_Y,train_Y)
TN1 = cfTrain[0,0]
FP1 = cfTrain[0,1]
FN1 = cfTrain[1,0]
TP1 = cfTrain[1,1]
Pre1 = TP1/(TP1+FP1) #Precision
Rec1 = TP1/(TP1+FN1) #Recall
F1_1 = 2*(Pre1*Rec1)/(Pre1+Rec1) # F1-score
cfTest =confusion_matrix(t_real_Y,test_Y)
TN2 = cfTest[0,0]
FP2 = cfTest[0,1]
FN2 = cfTest[1,0]
TP2 = cfTest[1,1]
Pre2 = TP2/(TP2+FP2) #Precision
Rec2 = TP2/(TP2+FN2) #Recall
F1_2 = 2*(Pre2*Rec2)/(Pre2+Rec2) # F1-score
cfVali =confusion_matrix(v_real_Y,vali_Y)
TN3 = cfVali[0,0]
FP3 = cfVali[0,1]
FN3 = cfVali[1,0]
TP3 = cfVali[1,1]
Pre3 = TP3/(TP3+FP3) #Precision
Rec3 = TP3/(TP3+FN3) #Recall
F1_3 = 2*(Pre3*Rec3)/(Pre2+Rec3) # F1-score
acc3 = (TP3+TN3)/(TN3+FP3+FN3+TP3)
csi3 = TP3/(TP3+FP3+FN3)
print(cfTrain)
print(cfTest)
print(cfVali)
#another score
acc1 = (TP1+TN1)/(TN1+FP1+FN1+TP1)
csi1 = TP1/(TP1+FP1+FN1)
acc2 = (TP2+TN2)/(TN2+FP2+FN2+TP2)
csi2 = TP2/(TP2+FP2+FN2)
print("--------- Train check -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_1,acc1,Pre1,Rec1,csi1))
print("--------- Verification -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_2,acc2,Pre2,Rec2,csi2))
print("--------- Validation -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_3,acc3,Pre3,Rec3,csi3))

[[7938 8298]
 [1934 7877]]
[[7176 4779]
 [4199 9717]]
[[3119 1816]
 [ 771 1349]]
--------- Train check -----------
F1:0.606250, acc:0.607172, pre:0.486986, rec:0.802874, csi:0.434977
--------- Verification -----------
F1:0.684007, acc:0.652971, pre:0.670323, rec:0.698261, csi:0.519765
--------- Validation -----------
F1:0.415133, acc:0.633310, pre:0.426224, rec:0.636321, csi:0.342734


In [3]:
# 6class ver.
# load dataset (pickle files)

input_path1 = 'C:/Users/LG/Desktop/frost_data/new3/6class/SMOTE'
cut_i = [0, 12, 24, 36, 48, 60, 72]
j = 0
for i in range(1,7):
    with open(f'{input_path1}/X_train_T{i}.pkl','rb') as f1:
        X_train_od = pickle.load(f1)
    with open(f'{input_path1}/Y_train_T{i}.pkl','rb') as f2:
        Y_train_od = pickle.load(f2) 
    if i == 1:
        X_train = X_train_od
        Y_train = Y_train_od
    else:
        X_train = np.vstack((X_train,X_train_od))
        Y_train = np.hstack((Y_train,Y_train_od))

X_train = np.delete(X_train, 1, axis = 1)
Y_train = Y_train.T

In [8]:
clf = RandomForestClassifier(n_estimators=130, max_depth=5, random_state = 0)
model = clf.fit(X_train, Y_train)
test_pb = model.predict(X_test)
vali_pb = model.predict(X_vali)
train_pb = model.predict(X_train2)

In [9]:
i = 0
j = 0
while i + 12 < test_pb.size:
    if 0 == sum(test_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_test[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        test_Y = frost_ck
        t_real_Y = real_ck
        j += 1
    else:
        test_Y = np.hstack((test_Y,frost_ck))
        t_real_Y = np.hstack((t_real_Y,real_ck))

i = 0
j = 0
while i + 12 < vali_pb.size:
    if 0 == sum(vali_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_vali[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        vali_Y = frost_ck
        v_real_Y = real_ck
        j += 1
    else:
        vali_Y = np.hstack((vali_Y,frost_ck))
        v_real_Y = np.hstack((v_real_Y,real_ck))
i = 0
j = 0
while i + 12 < train_pb.size:
    if 0 == sum(train_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_train2[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        train_Y = frost_ck
        tr_real_Y = real_ck
        j += 1
    else:
        train_Y = np.hstack((train_Y,frost_ck))
        tr_real_Y = np.hstack((tr_real_Y,real_ck))
cfTrain =confusion_matrix(tr_real_Y,train_Y)
TN1 = cfTrain[0,0]
FP1 = cfTrain[0,1]
FN1 = cfTrain[1,0]
TP1 = cfTrain[1,1]
Pre1 = TP1/(TP1+FP1) #Precision
Rec1 = TP1/(TP1+FN1) #Recall
F1_1 = 2*(Pre1*Rec1)/(Pre1+Rec1) # F1-score
cfTest =confusion_matrix(t_real_Y,test_Y)
TN2 = cfTest[0,0]
FP2 = cfTest[0,1]
FN2 = cfTest[1,0]
TP2 = cfTest[1,1]
Pre2 = TP2/(TP2+FP2) #Precision
Rec2 = TP2/(TP2+FN2) #Recall
F1_2 = 2*(Pre2*Rec2)/(Pre2+Rec2) # F1-score
cfVali =confusion_matrix(v_real_Y,vali_Y)
TN3 = cfVali[0,0]
FP3 = cfVali[0,1]
FN3 = cfVali[1,0]
TP3 = cfVali[1,1]
Pre3 = TP3/(TP3+FP3) #Precision
Rec3 = TP3/(TP3+FN3) #Recall
F1_3 = 2*(Pre3*Rec3)/(Pre2+Rec3) # F1-score
acc3 = (TP3+TN3)/(TN3+FP3+FN3+TP3)
csi3 = TP3/(TP3+FP3+FN3)
print(cfTrain)
print(cfTest)
print(cfVali)
#another score
acc1 = (TP1+TN1)/(TN1+FP1+FN1+TP1)
csi1 = TP1/(TP1+FP1+FN1)
acc2 = (TP2+TN2)/(TN2+FP2+FN2+TP2)
csi2 = TP2/(TP2+FP2+FN2)
print("--------- Train check -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_1,acc1,Pre1,Rec1,csi1))
print("--------- Verification -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_2,acc2,Pre2,Rec2,csi2))
print("--------- Validation -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_3,acc3,Pre3,Rec3,csi3))

[[7501 8735]
 [1844 7967]]
[[ 6946  5009]
 [ 3879 10037]]
[[2995 1940]
 [ 702 1418]]
--------- Train check -----------
F1:0.600988, acc:0.593850, pre:0.477009, rec:0.812048, csi:0.429581
--------- Verification -----------
F1:0.693115, acc:0.656449, pre:0.667088, rec:0.721256, csi:0.530357
--------- Validation -----------
F1:0.422838, acc:0.625514, pre:0.422275, rec:0.668868, csi:0.349261


In [18]:
i = 0
j = 0
while i + 12 < test_pb.size:
    if 0 == sum(test_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_test[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        test_Y = frost_ck
        t_real_Y = real_ck
        j += 1
    else:
        test_Y = np.hstack((test_Y,frost_ck))
        t_real_Y = np.hstack((t_real_Y,real_ck))

i = 0
j = 0
while i + 12 < vali_pb.size:
    if 0 == sum(vali_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_vali[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        vali_Y = frost_ck
        v_real_Y = real_ck
        j += 1
    else:
        vali_Y = np.hstack((vali_Y,frost_ck))
        v_real_Y = np.hstack((v_real_Y,real_ck))
i = 0
j = 0
while i + 12 < train_pb.size:
    if 0 == sum(train_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_train2[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        train_Y = frost_ck
        tr_real_Y = real_ck
        j += 1
    else:
        train_Y = np.hstack((train_Y,frost_ck))
        tr_real_Y = np.hstack((tr_real_Y,real_ck))
cfTrain =confusion_matrix(tr_real_Y,train_Y)
TN1 = cfTrain[0,0]
FP1 = cfTrain[0,1]
FN1 = cfTrain[1,0]
TP1 = cfTrain[1,1]
Pre1 = TP1/(TP1+FP1) #Precision
Rec1 = TP1/(TP1+FN1) #Recall
F1_1 = 2*(Pre1*Rec1)/(Pre1+Rec1) # F1-score
cfTest =confusion_matrix(t_real_Y,test_Y)
TN2 = cfTest[0,0]
FP2 = cfTest[0,1]
FN2 = cfTest[1,0]
TP2 = cfTest[1,1]
Pre2 = TP2/(TP2+FP2) #Precision
Rec2 = TP2/(TP2+FN2) #Recall
F1_2 = 2*(Pre2*Rec2)/(Pre2+Rec2) # F1-score
cfVali =confusion_matrix(v_real_Y,vali_Y)
TN3 = cfVali[0,0]
FP3 = cfVali[0,1]
FN3 = cfVali[1,0]
TP3 = cfVali[1,1]
Pre3 = TP3/(TP3+FP3) #Precision
Rec3 = TP3/(TP3+FN3) #Recall
F1_3 = 2*(Pre3*Rec3)/(Pre2+Rec3) # F1-score
acc3 = (TP3+TN3)/(TN3+FP3+FN3+TP3)
csi3 = TP3/(TP3+FP3+FN3)
print(cfTrain)
print(cfTest)
print(cfVali)
#another score
acc1 = (TP1+TN1)/(TN1+FP1+FN1+TP1)
csi1 = TP1/(TP1+FP1+FN1)
acc2 = (TP2+TN2)/(TN2+FP2+FN2+TP2)
csi2 = TP2/(TP2+FP2+FN2)
print("--------- Train check -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_1,acc1,Pre1,Rec1,csi1))
print("--------- Verification -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_2,acc2,Pre2,Rec2,csi2))
print("--------- Validation -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_3,acc3,Pre3,Rec3,csi3))

[[7474 8762]
 [1824 7987]]
[[ 6919  5036]
 [ 3765 10151]]
[[2803 2132]
 [ 634 1486]]
--------- Train check -----------
F1:0.601431, acc:0.593581, pre:0.476864, rec:0.814086, csi:0.430033
--------- Verification -----------
F1:0.697591, acc:0.659812, pre:0.668401, rec:0.729448, csi:0.535616
--------- Validation -----------
F1:0.420485, acc:0.607938, pre:0.410724, rec:0.700943, csi:0.349483


In [20]:
# 24class ver.
# load dataset (pickle files)

input_path1 = 'C:/Users/LG/Desktop/frost_data/new3/24class/SMOTE'
cut_i = range(0,73,3)
j = 0
for i in range(1,25):
    with open(f'{input_path1}/X_train_T{i}.pkl','rb') as f1:
        X_train_od = pickle.load(f1)
    with open(f'{input_path1}/Y_train_T{i}.pkl','rb') as f2:
        Y_train_od = pickle.load(f2) 
    if i == 1:
        X_train = X_train_od
        Y_train = Y_train_od
    else:
        X_train = np.vstack((X_train,X_train_od))
        Y_train = np.hstack((Y_train,Y_train_od))

X_train = np.delete(X_train, 1, axis = 1)
Y_train = Y_train.T

In [27]:
clf = RandomForestClassifier(n_estimators=130, max_depth=6, random_state = 0)
model = clf.fit(X_train, Y_train)
test_pb = model.predict(X_test)
vali_pb = model.predict(X_vali)
train_pb = model.predict(X_train2)

In [28]:
i = 0
j = 0
while i + 12 < test_pb.size:
    if 0 == sum(test_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_test[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        test_Y = frost_ck
        t_real_Y = real_ck
        j += 1
    else:
        test_Y = np.hstack((test_Y,frost_ck))
        t_real_Y = np.hstack((t_real_Y,real_ck))

i = 0
j = 0
while i + 12 < vali_pb.size:
    if 0 == sum(vali_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_vali[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        vali_Y = frost_ck
        v_real_Y = real_ck
        j += 1
    else:
        vali_Y = np.hstack((vali_Y,frost_ck))
        v_real_Y = np.hstack((v_real_Y,real_ck))
i = 0
j = 0
while i + 12 < train_pb.size:
    if 0 == sum(train_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_train2[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        train_Y = frost_ck
        tr_real_Y = real_ck
        j += 1
    else:
        train_Y = np.hstack((train_Y,frost_ck))
        tr_real_Y = np.hstack((tr_real_Y,real_ck))
cfTrain =confusion_matrix(tr_real_Y,train_Y)
TN1 = cfTrain[0,0]
FP1 = cfTrain[0,1]
FN1 = cfTrain[1,0]
TP1 = cfTrain[1,1]
Pre1 = TP1/(TP1+FP1) #Precision
Rec1 = TP1/(TP1+FN1) #Recall
F1_1 = 2*(Pre1*Rec1)/(Pre1+Rec1) # F1-score
cfTest =confusion_matrix(t_real_Y,test_Y)
TN2 = cfTest[0,0]
FP2 = cfTest[0,1]
FN2 = cfTest[1,0]
TP2 = cfTest[1,1]
Pre2 = TP2/(TP2+FP2) #Precision
Rec2 = TP2/(TP2+FN2) #Recall
F1_2 = 2*(Pre2*Rec2)/(Pre2+Rec2) # F1-score
cfVali =confusion_matrix(v_real_Y,vali_Y)
TN3 = cfVali[0,0]
FP3 = cfVali[0,1]
FN3 = cfVali[1,0]
TP3 = cfVali[1,1]
Pre3 = TP3/(TP3+FP3) #Precision
Rec3 = TP3/(TP3+FN3) #Recall
F1_3 = 2*(Pre3*Rec3)/(Pre2+Rec3) # F1-score
acc3 = (TP3+TN3)/(TN3+FP3+FN3+TP3)
csi3 = TP3/(TP3+FP3+FN3)
print(cfTrain)
print(cfTest)
print(cfVali)
#another score
acc1 = (TP1+TN1)/(TN1+FP1+FN1+TP1)
csi1 = TP1/(TP1+FP1+FN1)
acc2 = (TP2+TN2)/(TN2+FP2+FN2+TP2)
csi2 = TP2/(TP2+FP2+FN2)
print("--------- Train check -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_1,acc1,Pre1,Rec1,csi1))
print("--------- Verification -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_2,acc2,Pre2,Rec2,csi2))
print("--------- Validation -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_3,acc3,Pre3,Rec3,csi3))

[[7535 8701]
 [1745 8066]]
[[ 6758  5197]
 [ 3825 10091]]
[[2835 2100]
 [ 668 1452]]
--------- Train check -----------
F1:0.606968, acc:0.598956, pre:0.481064, rec:0.822138, csi:0.435717
--------- Verification -----------
F1:0.691070, acc:0.651270, pre:0.660060, rec:0.725137, csi:0.527965
--------- Validation -----------
F1:0.416335, acc:0.607654, pre:0.408784, rec:0.684906, csi:0.344076


In [29]:
# 36class ver.
# load dataset (pickle files)

input_path1 = 'C:/Users/LG/Desktop/frost_data/new3/36class/SMOTE'
cut_i = range(0,73,2)
j = 0
for i in range(1,37):
    with open(f'{input_path1}/X_train_T{i}.pkl','rb') as f1:
        X_train_od = pickle.load(f1)
    with open(f'{input_path1}/Y_train_T{i}.pkl','rb') as f2:
        Y_train_od = pickle.load(f2) 
    if i == 1:
        X_train = X_train_od
        Y_train = Y_train_od
    else:
        X_train = np.vstack((X_train,X_train_od))
        Y_train = np.hstack((Y_train,Y_train_od))

X_train = np.delete(X_train, 1, axis = 1)
Y_train = Y_train.T

In [36]:
clf = RandomForestClassifier(n_estimators=130, max_depth=5, random_state = 0)
model = clf.fit(X_train, Y_train)
test_pb = model.predict(X_test)
vali_pb = model.predict(X_vali)
train_pb = model.predict(X_train2)

In [37]:
i = 0
j = 0
while i + 12 < test_pb.size:
    if 0 == sum(test_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_test[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        test_Y = frost_ck
        t_real_Y = real_ck
        j += 1
    else:
        test_Y = np.hstack((test_Y,frost_ck))
        t_real_Y = np.hstack((t_real_Y,real_ck))

i = 0
j = 0
while i + 12 < vali_pb.size:
    if 0 == sum(vali_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_vali[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        vali_Y = frost_ck
        v_real_Y = real_ck
        j += 1
    else:
        vali_Y = np.hstack((vali_Y,frost_ck))
        v_real_Y = np.hstack((v_real_Y,real_ck))
i = 0
j = 0
while i + 12 < train_pb.size:
    if 0 == sum(train_pb[i:i+12]):
        frost_ck = 0
    else:
        frost_ck = 1
    if 0 == sum(Y_train2[i:i+12]):
        real_ck = 0
    else:
        real_ck = 1
    i += 12
    if j == 0:
        train_Y = frost_ck
        tr_real_Y = real_ck
        j += 1
    else:
        train_Y = np.hstack((train_Y,frost_ck))
        tr_real_Y = np.hstack((tr_real_Y,real_ck))
cfTrain =confusion_matrix(tr_real_Y,train_Y)
TN1 = cfTrain[0,0]
FP1 = cfTrain[0,1]
FN1 = cfTrain[1,0]
TP1 = cfTrain[1,1]
Pre1 = TP1/(TP1+FP1) #Precision
Rec1 = TP1/(TP1+FN1) #Recall
F1_1 = 2*(Pre1*Rec1)/(Pre1+Rec1) # F1-score
cfTest =confusion_matrix(t_real_Y,test_Y)
TN2 = cfTest[0,0]
FP2 = cfTest[0,1]
FN2 = cfTest[1,0]
TP2 = cfTest[1,1]
Pre2 = TP2/(TP2+FP2) #Precision
Rec2 = TP2/(TP2+FN2) #Recall
F1_2 = 2*(Pre2*Rec2)/(Pre2+Rec2) # F1-score
cfVali =confusion_matrix(v_real_Y,vali_Y)
TN3 = cfVali[0,0]
FP3 = cfVali[0,1]
FN3 = cfVali[1,0]
TP3 = cfVali[1,1]
Pre3 = TP3/(TP3+FP3) #Precision
Rec3 = TP3/(TP3+FN3) #Recall
F1_3 = 2*(Pre3*Rec3)/(Pre2+Rec3) # F1-score
acc3 = (TP3+TN3)/(TN3+FP3+FN3+TP3)
csi3 = TP3/(TP3+FP3+FN3)
print(cfTrain)
print(cfTest)
print(cfVali)
#another score
acc1 = (TP1+TN1)/(TN1+FP1+FN1+TP1)
csi1 = TP1/(TP1+FP1+FN1)
acc2 = (TP2+TN2)/(TN2+FP2+FN2+TP2)
csi2 = TP2/(TP2+FP2+FN2)
print("--------- Train check -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_1,acc1,Pre1,Rec1,csi1))
print("--------- Verification -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_2,acc2,Pre2,Rec2,csi2))
print("--------- Validation -----------")
print("F1:{:2f}, acc:{:2f}, pre:{:2f}, rec:{:2f}, csi:{:2f}".format(F1_3,acc3,Pre3,Rec3,csi3))

[[7085 9151]
 [1696 8115]]
[[ 6648  5307]
 [ 3370 10546]]
[[2521 2414]
 [ 533 1587]]
--------- Train check -----------
F1:0.599402, acc:0.583560, pre:0.469999, rec:0.827133, csi:0.427961
--------- Verification -----------
F1:0.708522, acc:0.664605, pre:0.665237, rec:0.757833, csi:0.548614
--------- Validation -----------
F1:0.420034, acc:0.582282, pre:0.396651, rec:0.748585, csi:0.350022
